# Global Emissions by Country

## Introduction

In this assignment I'm going to analyse the global emission dataset for the period between 2000 and 2012 and I'll be trying to answer some questions about the ***Top countries in emission***, ***CO2 emission over the years***, and ***if there is a specific decrease in emission or not?***
to begin with that I need to highlight here what emissions I'm considering and thier categories to just clarify more what are these appreviations for, ***as follows:***

Emissions are calculated for the following substances: 1) Direct greenhouse gases: Carbon Dioxide (CO2), Methane (CH4), Nitrous Oxide (N2O), Hydrofluorocarbons (HFC-23, 32, 125, 134a, 143a, 152a, 227ea, 236fa, 245fa, 365mfc, 43-10-mee), Perfluorocarbons (PFCs: CF4, C2F6, C3F8, c-C4F8, C4F10, C5F12, C6F14, C7F16), Sulfur Hexafluoride (SF6), Nitrogen Trifluoride (NF3) and Sulfuryl Fluoride (SO2F2); 2) Ozone precursor gases: Carbon Monoxide (CO), Nitrogen Oxides (NOx), Non-Methane Volatile Organic Compounds (NMVOC) and Methane (CH4). 3) Acidifying gases: Ammonia (NH3), Nitrogen oxides (NOx) and Sulfur Dioxide (SO2). 4) Primary particulates: Fine Particulate Matter (PM10) - Carbonaceous speciation (BC , OC) is under progress. 5) Stratospheric Ozone Depleting Substances: Chlorofluorocarbons (CFC-11, 12, 113, 114, 115), Halons (1211, 1301, 2402), Hydrochlorofluorocarbons (HCFC-22, 124, 141b, 142b), Carbon Tetrachloride (CCl4), Methyl Bromide (CH3Br) and Methyl Chloroform (CH3CCl2). Emissions (EM) for a country C are calculated for each compound x on an annual basis (y) and sector wise (for i sectors, multiplying on the one hand the country-specific activity data (AD), quantifying the human activity for each of the i sectors, with the mix of j technologies (TECH) for each sector i, and with their abatement percentage by one of the k end-of-pipe (EOP) measures for each technology j, and on the other hand the country-specific emission factor (EF) for each sector i and technology j with relative reduction (RED) of the uncontrolled emission by installed abatement measure k. Emissions in are calculated by individual countries using country-specific information. The countries are organized in different world regions for illustration purposes. Emissions of some small countries are presented together with other countries depending on country definition and availability of activity statistics

# Section1 (Setup)
    1.1. Libraries Import.
    1.2. myFunctions.
    1.3. Spark session establishment

### 1.1. Libraries Import

In [84]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from IPython.display import display, Markdown
from pyspark.sql.functions import count, col, first, lit,year, month,udf, dayofmonth,when,countDistinct,round,sum,expr,substring
from IPython.display import display, Markdown
from pyspark.sql.functions import max
from pyspark.sql.types import IntegerType
from functools import reduce

### 1.2. myFunctions

In [85]:
def superSum(*cols):
    return reduce(lambda a, b: a + b, cols)

add = udf(superSum)


### 1.3. Establishing spark session 

In [86]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Section2 (Reading CSV & data discovery)
    2.1. Reading Data/printing Schema
    2.2. Data discovery.

### 2.1. Reading Data/printing Schema

In [87]:
gemissionDF = spark.read \
             .option("inferSchema", "true") \
             .option("header", "true") \
             .option("sep", ";") \
             .csv("global-emissions-by-country-2013.csv")
gemissionDF.printSchema()
display(Markdown("This global emission DataFrame has **%d rows**." % gemissionDF.count()))

root
 |-- COUNTRY_ID: string (nullable = true)
 |-- COUNTRY_NAME: string (nullable = true)
 |-- COUNTRY_NOTES: string (nullable = true)
 |-- COUNTRY_REGIONID: string (nullable = true)
 |-- DATE: timestamp (nullable = true)
 |-- FREQUENCY: string (nullable = true)
 |-- INDICATOR_ID: string (nullable = true)
 |-- INDICATOR_NAME: string (nullable = true)
 |-- IPCC_ID: string (nullable = true)
 |-- IPCC_NAME: string (nullable = true)
 |-- VALUE: double (nullable = true)



This global emission DataFrame has **1826762 rows**.

#### Now, let's check randomly two records

In [88]:
gemissionDF.cache() 
gemissionDF.sample(False, 0.1).take(2)

[Row(COUNTRY_ID='EH', COUNTRY_NAME='Western Sahara', COUNTRY_NOTES='Northern_Africa', COUNTRY_REGIONID='EH', DATE=datetime.datetime(1971, 1, 1, 0, 0), FREQUENCY='Annual', INDICATOR_ID='KN.A5', INDICATOR_NAME='N2O', IPCC_ID='6B', IPCC_NAME='Wastewater handling', VALUE=0.0639142),
 Row(COUNTRY_ID='EH', COUNTRY_NAME='Western Sahara', COUNTRY_NOTES='Northern_Africa', COUNTRY_REGIONID='EH', DATE=datetime.datetime(2004, 1, 1, 0, 0), FREQUENCY='Annual', INDICATOR_ID='KN.A5', INDICATOR_NAME='N2O', IPCC_ID='7B', IPCC_NAME='Indirect N2O from non-agricultural NOx', VALUE=0.00469136)]

#### Since we have 1.9 M records, i'm going to reduce it to around 500 K by choosing only all data from the year 2000

In [89]:
subDF = gemissionDF.where(year("date") >= 2000)
#subDF = gemissionDF
subDF.cache() 

DataFrame[COUNTRY_ID: string, COUNTRY_NAME: string, COUNTRY_NOTES: string, COUNTRY_REGIONID: string, DATE: timestamp, FREQUENCY: string, INDICATOR_ID: string, INDICATOR_NAME: string, IPCC_ID: string, IPCC_NAME: string, VALUE: double]

In [90]:
display(Markdown("This global emission DataFrame has **%d rows**. from 2000 till 2012" % subDF.count()))

This global emission DataFrame has **518789 rows**. from 2000 till 2012

#### Now, let's free some memory ;) 

In [91]:
gemissionDF.unpersist()

DataFrame[COUNTRY_ID: string, COUNTRY_NAME: string, COUNTRY_NOTES: string, COUNTRY_REGIONID: string, DATE: timestamp, FREQUENCY: string, INDICATOR_ID: string, INDICATOR_NAME: string, IPCC_ID: string, IPCC_NAME: string, VALUE: double]

### 2.2 Data discovery.

In [92]:
print ("emmission value summary:")
subDF.select("VALUE").summary().show()

print("Checking also if there is nulls on column VALUE:")
subDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["VALUE"]]).show()

print ("COUNTRY_ID,COUNTRY_NAME,COUNTRY_NOTES,COUNTRY_REGIONID,DATE summary:")
subDF.select("COUNTRY_ID","COUNTRY_NAME","COUNTRY_NOTES","COUNTRY_REGIONID","DATE").summary().show()

print ("FREQUENCY,INDICATOR_ID,INDICATOR_NAME,IPCC_ID,IPCC_NAME summary:")
subDF.select("FREQUENCY","INDICATOR_ID","INDICATOR_NAME","IPCC_ID","IPCC_NAME").summary().show()


print("Checking amount of distinct values in columns COUNTRY_ID,COUNTRY_NAME,COUNTRY_NOTES,COUNTRY_REGIONID, \
      DATE,FREQUENCY,INDICATOR_ID,INDICATOR_NAME,IPCC_ID,IPCC_NAME:")
subDF.select([countDistinct(c).alias(c) 
              for c in ["COUNTRY_ID","COUNTRY_NAME","COUNTRY_NOTES","COUNTRY_REGIONID","DATE","FREQUENCY",
                        "INDICATOR_ID","INDICATOR_NAME","IPCC_ID","IPCC_NAME"]]).show()

print("Checking also if there is nulls on other columns in the dataset :")
subDF.select([count(when(col(c).isNull(), c))
              .alias(c) for c in ["COUNTRY_ID","COUNTRY_NAME","COUNTRY_NOTES","COUNTRY_REGIONID",
                                  "DATE","FREQUENCY","INDICATOR_ID","INDICATOR_NAME","IPCC_ID","IPCC_NAME"]]).show()


emmission value summary:
+-------+------------------+
|summary|             VALUE|
+-------+------------------+
|  count|            518789|
|   mean|3251.4186273294995|
| stddev| 73873.94031694348|
|    min|         -496000.0|
|    25%|         0.0162442|
|    50%|           0.47762|
|    75%|      11.815299832|
|    max|  9918456.19197339|
+-------+------------------+

Checking also if there is nulls on column VALUE:
+-----+
|VALUE|
+-----+
|    0|
+-----+

COUNTRY_ID,COUNTRY_NAME,COUNTRY_NOTES,COUNTRY_REGIONID,DATE summary:
+-------+----------+------------+--------------+----------------+
|summary|COUNTRY_ID|COUNTRY_NAME| COUNTRY_NOTES|COUNTRY_REGIONID|
+-------+----------+------------+--------------+----------------+
|  count|    518789|      518789|        511327|          514037|
|   mean|      null|        null|          null|            null|
| stddev|      null|        null|          null|            null|
|    min|        AE| Afghanistan|     Asia-Stan|              AE|
|    

In [93]:
print ("Most and least frequent occurrences for COUNTRY_NAME, COUNTRY_NOTES, COUNTRY_REGIONID and INDICATOR_NAME columns:")
COUNTRY_NAMEDF = subDF.groupBy("COUNTRY_NAME").agg(count(lit(1)).alias("Total"))
COUNTRY_NOTESDF   = subDF.groupBy("COUNTRY_NOTES").agg(count(lit(1)).alias("Total"))
COUNTRY_REGIONIDDF    = subDF.groupBy("COUNTRY_REGIONID").agg(count(lit(1)).alias("Total"))
INDICATOR_NAMEDF      = subDF.groupBy("INDICATOR_NAME").agg(count(lit(1)).alias("Total"))

leastFreqCOUNTRY_NAME    = COUNTRY_NAMEDF.orderBy(col("Total").asc()).first()
mostFreqCOUNTRY_NAME    = COUNTRY_NAMEDF.orderBy(col("Total").desc()).first()
leastFreqCOUNTRY_NOTES      = COUNTRY_NOTESDF.orderBy(col("Total").asc()).first()
mostFreqCOUNTRY_NOTES      = COUNTRY_NOTESDF.orderBy(col("Total").desc()).first()
leastFreqCOUNTRY_REGIONID       = COUNTRY_REGIONIDDF.orderBy(col("Total").asc()).first()
mostFreqCOUNTRY_REGIONID        = COUNTRY_REGIONIDDF.orderBy(col("Total").desc()).first()
leastFreqINDICATOR_NAME         = INDICATOR_NAMEDF.orderBy(col("Total").asc()).first()
mostFreqINDICATOR_NAME          = INDICATOR_NAMEDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqCOUNTRY_NAME", "mostFreqCOUNTRY_NAME", "leastFreqCOUNTRY_NOTES", "mostFreqCOUNTRY_NOTES", \
       "%s (%d occurrences)" % (leastFreqCOUNTRY_NAME["COUNTRY_NAME"], leastFreqCOUNTRY_NAME["Total"]), \
       "%s (%d occurrences)" % (mostFreqCOUNTRY_NAME["COUNTRY_NAME"], mostFreqCOUNTRY_NAME["Total"]), \
       "%s (%d occurrences)" % (leastFreqCOUNTRY_NOTES["COUNTRY_NOTES"], leastFreqCOUNTRY_NOTES["Total"]), \
       "%s (%d occurrences)" % (mostFreqCOUNTRY_NOTES["COUNTRY_NOTES"], mostFreqCOUNTRY_NOTES["Total"]))))
display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqCOUNTRY_REGIONID", "mostFreqCOUNTRY_REGIONID", "leastFreqINDICATOR_NAME", "mostFreqINDICATOR_NAME", \
       "%s (%d occurrences)" % (leastFreqCOUNTRY_REGIONID["COUNTRY_REGIONID"], leastFreqCOUNTRY_REGIONID["Total"]), \
       "%s (%d occurrences)" % (mostFreqCOUNTRY_REGIONID["COUNTRY_REGIONID"], mostFreqCOUNTRY_REGIONID["Total"]), \
       "%s (%d occurrences)" % (leastFreqINDICATOR_NAME["INDICATOR_NAME"], leastFreqINDICATOR_NAME["Total"]), \
       "%s (%d occurrences)" % (mostFreqINDICATOR_NAME["INDICATOR_NAME"], mostFreqINDICATOR_NAME["Total"]))))


print ("Most and least frequent occurrences for IPCC_NAME columns:")
IPCC_NAMEDF = subDF.groupBy("IPCC_NAME").agg(count(lit(1)).alias("Total"))

leastFreqIPCC_NAME   = IPCC_NAMEDF.orderBy(col("Total").asc()).first()
mostFreqIPCC_NAME    = IPCC_NAMEDF.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s |
|----|----|
| %s | %s | 
""" % ("leastFreqIPCC_NAME", "mostFreqIPCC_NAME", \
       "%s (%d occurrences)" % (leastFreqIPCC_NAME["IPCC_NAME"], leastFreqIPCC_NAME["Total"]), \
       "%s (%d occurrences)" % (mostFreqIPCC_NAME["IPCC_NAME"], mostFreqIPCC_NAME["Total"]))))



Most and least frequent occurrences for COUNTRY_NAME, COUNTRY_NOTES, COUNTRY_REGIONID and INDICATOR_NAME columns:



| leastFreqCOUNTRY_NAME | mostFreqCOUNTRY_NAME | leastFreqCOUNTRY_NOTES | mostFreqCOUNTRY_NOTES |
|----|----|----|----|
| Norfolk Island (30 occurrences) | Australia (3884 occurrences) | Int. Aviation (303 occurrences) | Rest Central America (65640 occurrences) |



| leastFreqCOUNTRY_REGIONID | mostFreqCOUNTRY_REGIONID | leastFreqINDICATOR_NAME | mostFreqINDICATOR_NAME |
|----|----|----|----|
| PN (30 occurrences) | None (4752 occurrences) | C5F12 (14 occurrences) | N2O (49266 occurrences) |


Most and least frequent occurrences for IPCC_NAME columns:



| leastFreqIPCC_NAME | mostFreqIPCC_NAME |
|----|----|
| Fugitive emissions from solid fuels (2) (22 occurrences) | Total (48001 occurrences) | 


# Section3 (My Insights)
3.1. Which is the most emissions polluting the earth? <br>
3.2. Which countries generates most of the world’s CO2 emissions?<br>
3.3 Is there a change in emission since 2000 ?

### 3.1 Which is the most emissions polluting the earth?

TOP20

In [94]:
display(Markdown("TOP20 **most emmissions types poluted the earth** (in Megatons):"))
mostIndicatorsDF = subDF.groupBy("INDICATOR_ID","INDICATOR_NAME") \
                       .agg((round(sum("VALUE")/1000000,2)).alias("sumEmission(Mton)")) \
                       .sort("sumEmission(Mton)", ascending=False) \
                       .show()

TOP20 **most emmissions types poluted the earth** (in Megatons):

+------------+--------------------+-----------------+
|INDICATOR_ID|      INDICATOR_NAME|sumEmission(Mton)|
+------------+--------------------+-----------------+
|       KN.A2|CO2 excluding sho...|          1256.98|
|       KN.A3|CO2 organic carbo...|           386.86|
|       KN.G3|                  CO|            12.75|
|       KN.A4|                 CH4|              7.7|
|       KN.A6|       HFCs (GWP100)|             7.41|
|      KN.A32|               NMVOC|             2.48|
|       KN.G7|                 NOx|             2.32|
|      KN.G11|                 SO2|             2.04|
|      KN.A18|       PFCs (GWP100)|             1.63|
|       KN.G6|NMVOC_fossil  (NM...|             1.54|
|       KN.G5|NMVOC_bio (NMVOC ...|              1.3|
|       KN.G9|                PM10|             1.28|
|       KN.G4|                 NH3|             1.12|
|      KN.G10|               PM2.5|             0.82|
|       KN.G8|                  OC|             0.27|
|       KN.A5|              

#### Now it can be seen that CO2 (KN.A2, KN.A3) are both the most type that is affecting the whole earth, Now let's see the most two countries affecting the earth as well but only for ****CO2****

### 3.2 Which countries generates most of the world’s CO2 emissions?


****noting that we'll cobmine both Indicators (KN.A2 & KN.A3).***

In [95]:
display(Markdown("TOP10 **'COUNTRY-Wise' Most to lowest emissions** (in Megatons):"))

mostCo2DF = subDF.filter("INDICATOR_ID == 'KN.A2' or INDICATOR_ID == 'KN.A3'") \
            .groupBy("COUNTRY_NAME") \
            .agg((round(sum("VALUE")/1000000,2)).alias("sumEmission(Mton)")) \
            .sort("sumEmission(Mton)", ascending=False) \
            .show(20)


TOP10 **'COUNTRY-Wise' Most to lowest emissions** (in Megatons):

+--------------------+-----------------+
|        COUNTRY_NAME|sumEmission(Mton)|
+--------------------+-----------------+
|               China|           256.69|
|       United States|            221.3|
|               India|             81.6|
|  Russian Federation|            76.21|
|              Brazil|            63.47|
|           Indonesia|            62.23|
|               Japan|            51.56|
|             Germany|            39.39|
|Central African R...|             33.4|
|Congo_the Democra...|            32.01|
|              Canada|            29.58|
|           Australia|            24.92|
|       Int. Shipping|            24.31|
|             Nigeria|            23.96|
|               Sudan|            23.26|
|      United Kingdom|            23.11|
|  Korea, Republic of|             21.5|
|               Italy|             20.9|
|Iran, Islamic Rep...|            20.15|
|       Int. Aviation|            18.19|
+--------------------+-----------------+
only showing top

**From the above table, It Can be seen that China,US, and India are the most countries that produces CO2 on earth in the period from 2000 till 2012**

In [96]:
sumCO2Emission = subDF.groupBy().agg(round(sum("VALUE"),2)).collect()
NEWDF = subDF.select("COUNTRY_NAME","COUNTRY_NOTES","COUNTRY_REGIONID","date","IPCC_NAME","INDICATOR_NAME","VALUE")
NEWDF.cache()
display(Markdown("TOP20 **'INDICATOR_NAME-Wise' Most to lowest emissions** (in Megatons):"))
NEWDF.select("INDICATOR_NAME","VALUE")\
                      .groupBy("INDICATOR_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("INDICATOR_NAME",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                       .sort("Ratio%", ascending=False) \
                       .show()

display(Markdown("TOP20 **'COUNTRY-Wise' Most to lowest emissions** (in Megatons):"))
NEWDF.select("COUNTRY_NAME","VALUE")\
                     .groupBy("COUNTRY_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("COUNTRY_NAME",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False) \
                     .show()

TOP20 **'INDICATOR_NAME-Wise' Most to lowest emissions** (in Megatons):

+--------------------+-----------------+------+
|      INDICATOR_NAME|sumEmission(Mton)|Ratio%|
+--------------------+-----------------+------+
|CO2 excluding sho...|          1256.98| 74.52|
|CO2 organic carbo...|           386.86| 22.93|
|                  CO|            12.75|  0.76|
|                 CH4|              7.7|  0.46|
|       HFCs (GWP100)|             7.41|  0.44|
|               NMVOC|             2.48|  0.15|
|                 NOx|             2.32|  0.14|
|                 SO2|             2.04|  0.12|
|       PFCs (GWP100)|             1.63|   0.1|
|NMVOC_fossil  (NM...|             1.54|  0.09|
|                PM10|             1.28|  0.08|
|NMVOC_bio (NMVOC ...|              1.3|  0.08|
|                 NH3|             1.12|  0.07|
|               PM2.5|             0.82|  0.05|
|                  OC|             0.27|  0.02|
|                  BC|              0.1|  0.01|
|                 N2O|              0.2|  0.01|
|               C4F10|              0.0|

TOP20 **'COUNTRY-Wise' Most to lowest emissions** (in Megatons):

+--------------------+-----------------+------+
|        COUNTRY_NAME|sumEmission(Mton)|Ratio%|
+--------------------+-----------------+------+
|               China|           265.14| 15.72|
|       United States|           227.15| 13.47|
|               India|            85.26|  5.05|
|  Russian Federation|            78.08|  4.63|
|              Brazil|            65.14|  3.86|
|           Indonesia|            63.68|  3.78|
|               Japan|            52.73|  3.13|
|             Germany|            39.94|  2.37|
|Central African R...|            33.49|  1.99|
|Congo_the Democra...|            32.24|  1.91|
|              Canada|            30.31|   1.8|
|           Australia|            25.31|   1.5|
|       Int. Shipping|             24.9|  1.48|
|             Nigeria|            24.86|  1.47|
|               Sudan|            23.48|  1.39|
|      United Kingdom|            23.51|  1.39|
|  Korea, Republic of|            21.92|   1.3|
|               Italy|            21.25|

**Now, it can be seen that CO2 (KN.A2, KN.A3) are both the most type that is affecting the whole earth, so let's see the most two countries affecting the earth as well but only for CO2, both together are formulating 97.4% of the overall emissions on earth for the period 2000 till 2012!, <br> On the other hand, country wise,**
**from the above table, It Can be seen that China, US, India, Russia and Brazil  are the most countries that produces CO2 on earth with almost 44% of the over all emission in the period from 2000 till 2012**

In [97]:
display(Markdown("TOP10 **'COUNTRY_NOTES-Wise' Most to lowest emissions** (in Megatons):"))
NEWDF.select("COUNTRY_NOTES","VALUE")\
                     .groupBy("COUNTRY_NOTES") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("COUNTRY_NOTES",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False) \
                     .show(10)

display(Markdown("TOP10 **'COUNTRY_REGION-Wise' Most to lowest emissions** (in Megatons):"))
NEWDF.select("COUNTRY_REGIONID","VALUE")\
                     .groupBy("COUNTRY_REGIONID") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("COUNTRY_REGIONID",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False) \
                     .show(10)

display(Markdown("TOP10 **'Year-Wise' Most to lowest year emissions** (in Megatons):"))
NEWDF.select(year("date").alias("Year"),"VALUE")\
                     .groupBy("Year") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("Year",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False) \
                     .show(10)

display(Markdown("TOP10**'Year-Wise' Most to lowest year emissions** by Country (in Megatons):"))
NEWDF.select("COUNTRY_NAME",year("date").alias("Year"),"VALUE")\
                     .groupBy("Year","COUNTRY_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("Year","COUNTRY_NAME",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False) \
                     .show(10)
                     

TOP10 **'COUNTRY_NOTES-Wise' Most to lowest emissions** (in Megatons):

+-----------------+-----------------+------+
|    COUNTRY_NOTES|sumEmission(Mton)|Ratio%|
+-----------------+-----------------+------+
|             null|           295.45| 17.52|
|              USA|           227.15| 13.47|
|      OECD_Europe|           160.29|   9.5|
|   Western_Africa|           129.28|  7.66|
|          India +|           103.76|  6.15|
|         Russia +|            79.92|  4.74|
|      Indonesia +|            65.04|  3.86|
|           Brazil|            65.14|  3.86|
|Southeastern Asia|            62.19|  3.69|
|      Middle_East|            60.17|  3.57|
+-----------------+-----------------+------+
only showing top 10 rows



TOP10 **'COUNTRY_REGION-Wise' Most to lowest emissions** (in Megatons):

+----------------+-----------------+------+
|COUNTRY_REGIONID|sumEmission(Mton)|Ratio%|
+----------------+-----------------+------+
|              CN|           265.14| 15.72|
|              US|           227.15| 13.47|
|              IN|            85.26|  5.05|
|              RU|            78.08|  4.63|
|              BR|            65.14|  3.86|
|              ID|            63.68|  3.78|
|              JP|            52.73|  3.13|
|            null|            45.52|   2.7|
|              DE|            39.94|  2.37|
|              CF|            33.49|  1.99|
+----------------+-----------------+------+
only showing top 10 rows



TOP10 **'Year-Wise' Most to lowest year emissions** (in Megatons):

+----+-----------------+------+
|Year|sumEmission(Mton)|Ratio%|
+----+-----------------+------+
|2007|           181.45| 10.76|
|2008|            167.5|  9.93|
|2006|           164.32|  9.74|
|2004|           154.33|  9.15|
|2005|           148.99|  8.83|
|2003|           143.42|   8.5|
|2001|           142.84|  8.47|
|2002|           139.75|  8.28|
|2000|           134.08|  7.95|
|2011|            79.46|  4.71|
+----+-----------------+------+
only showing top 10 rows



TOP10**'Year-Wise' Most to lowest year emissions** by Country (in Megatons):

+----+-------------+-----------------+------+
|Year| COUNTRY_NAME|sumEmission(Mton)|Ratio%|
+----+-------------+-----------------+------+
|2007|        China|            27.65|  1.64|
|2006|        China|            25.67|  1.52|
|2004|        China|            25.21|  1.49|
|2003|United States|            24.23|  1.44|
|2007|United States|            23.77|  1.41|
|2008|        China|            23.49|  1.39|
|2008|United States|            23.22|  1.38|
|2000|United States|            22.48|  1.33|
|2011|        China|            21.02|  1.25|
|2001|United States|            20.98|  1.24|
+----+-------------+-----------------+------+
only showing top 10 rows



### 3.3 Is there a change in emission since 2000 ?

In [98]:
display(Markdown("TOP20 **Country-wise** over year from 2000 till 2012(in Megatons):"))
yearDF = NEWDF.select("COUNTRY_NAME",year("date").alias("Year"),"VALUE")\
                     .groupBy("Year","COUNTRY_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("Year",'COUNTRY_NAME',round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False)                     

yearPivot = yearDF.groupBy("COUNTRY_NAME").pivot("Year").sum("sumEmission(Mton)")
yearPivot = yearPivot.fillna(0, subset=yearPivot.columns[1:15])

yearPivot.withColumn('TOTAL', round(add(*[yearPivot[x] for x in yearPivot.columns[1:14]]),2)).withColumn('TOTAL%', \
        round((add(*[yearPivot[x] for x in yearPivot.columns[1:14]]))/(sumCO2Emission[0][0]/1000000)*100,2)) \
                    .sort("TOTAL", ascending=False) .show()


TOP20 **Country-wise** over year from 2000 till 2012(in Megatons):

+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
|        COUNTRY_NAME| 2000| 2001| 2002| 2003| 2004| 2005| 2006| 2007| 2008| 2009| 2010| 2011| 2012| TOTAL|TOTAL%|
+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
|               China|16.32|16.39|16.02|16.65|25.21| 19.2|25.67|27.65|23.49|18.86|17.79|21.02|20.89|265.16| 15.72|
|       United States|22.48|20.98|18.09|24.23| 18.1|18.68|17.71|23.77|23.22|11.12| 7.92|11.33| 9.51|227.14| 13.47|
|               India| 6.14| 7.83| 6.58| 6.52| 8.31| 6.39| 7.68| 8.84|  6.0| 5.29|  5.3| 4.43| 5.95| 85.26|  5.05|
|  Russian Federation| 4.06| 7.41| 8.22| 7.16| 7.58|  7.4|  7.4| 7.64| 7.54| 2.99| 3.46| 3.57| 3.64| 78.07|  4.63|
|              Brazil| 5.68| 6.39| 6.66| 6.75| 7.65| 7.93| 5.26| 5.36| 5.46|  1.9| 2.07|  2.0| 2.02| 65.13|  3.86|
|           Indonesia| 4.52| 5.22| 5.97|  4.4| 6.15| 8.75|10.75|  5.7| 6.58| 1.3

#### Country-Wise
**We can see from the above that the united states of America were at 1st place in emission till 2004 as China took the lead till now in polluting the whole world, 
there is also something happened in 2009 as there were a big drop by almost 50% in all countries and I think this is related to the economic crisis happened in 2008.**

#### Now, let's also see if something has happened from the CO2 wheather it has been reduced across all countries year-wise



In [99]:
display(Markdown("TOP5 **Indicators** over year from 2000 till 2012(in Megatons):"))
yearDF = NEWDF.select("INDICATOR_NAME",year("date").alias("Year"),"VALUE")\
                     .groupBy("Year","INDICATOR_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .withColumn('IND_NAME_S', substring('INDICATOR_NAME', 1, 5)) \
                     .select("Year",'IND_NAME_S',round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False)                     

yearPivot = yearDF.groupBy("IND_NAME_S").pivot("Year").sum("sumEmission(Mton)") \
                  .fillna(0, subset=yearPivot.columns[1:15])

yearPivot.withColumn('TOTAL', round(add(*[yearPivot[x] for x in yearPivot.columns[1:14]]),2)).withColumn('TOTAL%', \
        round((add(*[yearPivot[x] for x in yearPivot.columns[1:14]]))/(sumCO2Emission[0][0]/1000000)*100,2)) \
                    .sort("TOTAL", ascending=False) .show(5)


TOP5 **Indicators** over year from 2000 till 2012(in Megatons):

+----------+-----+------+------+------+------+-----+------+------+------+-----+-----+-----+-----+-------+------+
|IND_NAME_S| 2000|  2001|  2002|  2003|  2004| 2005|  2006|  2007|  2008| 2009| 2010| 2011| 2012|  TOTAL|TOTAL%|
+----------+-----+------+------+------+------+-----+------+------+------+-----+-----+-----+-----+-------+------+
|     CO2 e|99.82|104.55|101.09|106.09|116.75|109.7|119.54|127.62|118.62|61.59|59.44|66.27|65.89|1256.97| 74.52|
|     CO2 o|30.71| 34.76| 34.84| 33.48| 33.55|35.22| 40.58| 49.46|  44.4|12.18|12.35|12.56|12.79| 386.88| 22.94|
|        CO| 1.16|  1.17|  1.15|  1.16|  1.13| 1.11|  1.12|  1.15|   1.2|  1.2| 1.19|  0.0|  0.0|  12.74|  0.76|
|       CH4| 0.49|  0.53|  0.58|  0.56|  0.61|  0.6|  0.62|  0.63|  0.58| 0.59| 0.65| 0.62| 0.66|   7.72|  0.46|
|     HFCs |  0.5|  0.48|   0.7|  0.73|  0.86| 0.89|  0.98|  1.08|  1.19|  0.0|  0.0|  0.0|  0.0|   7.41|  0.44|
+----------+-----+------+------+------+------+-----+------+------+------+-----+-----+-----+-----

#### It can be seen from the above table that CO2 in both types are decreased dramatically to half of the percentage recorded in 2000, however, there is a slight increase in CH4 which is more dangerouse even with small doses to our earth, and to really imagine how CH4 is problematic to earth just imagine that its impact is 34 times greater than CO2 over a 100-year period, according to the latest IPCC Assessment Report.


***Note:***
Methane is a greenhouse gas as is carbon dioxide. Human activity has increased the amount of methane in the atmosphere, contributing to climate change. Methane is particularly problematic as its impact is 34 times greater than CO2 over a 100-year period, according to the latest IPCC Assessment Report. A significant source of human-made methane emissions is fossil fuel production. For example, methane is a key by-product of the rapidly rising global extraction and processing of natural gas. Other top sources of methane come from the digestive process of livestock and from landfills, which emit it as waste decomposes.

In [100]:
display(Markdown("TOP20 **emission sectors** over year from 2000 till 2012(in Megatons):"))
yearDF = NEWDF.select("IPCC_NAME",year("date").alias("Year"),"VALUE")\
                     .groupBy("Year","IPCC_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .withColumn('IND_NAME_S', substring('IPCC_NAME', 1, 35)) \
                     .select("Year",'IND_NAME_S',round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort("Ratio%", ascending=False)                     

yearPivot = yearDF.groupBy("IND_NAME_S").pivot("Year").sum("sumEmission(Mton)") \
                  .fillna(0, subset=yearPivot.columns[1:15])

yearPivot.withColumn('TOTAL', round(add(*[yearPivot[x] for x in yearPivot.columns[1:14]]),2)).withColumn('TOTAL%', \
        round((add(*[yearPivot[x] for x in yearPivot.columns[1:14]]))/(sumCO2Emission[0][0]/1000000)*100,2)) \
                    .sort("TOTAL", ascending=False) .show()

TOP20 **emission sectors** over year from 2000 till 2012(in Megatons):

+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
|          IND_NAME_S| 2000| 2001| 2002| 2003| 2004| 2005| 2006| 2007| 2008| 2009| 2010| 2011| 2012| TOTAL|TOTAL%|
+--------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
|excluding carbon ...|64.16|71.41|64.03|64.64|70.81|66.16|80.61|91.56|79.72|  0.0|  0.0|  0.0|  0.0| 653.1| 38.72|
|               Total|32.82|33.03|33.67|35.09|36.65|37.79|38.92| 40.4| 40.8|39.57|41.52|41.54| 41.9| 493.7| 29.27|
|Public electricit...| 6.33| 7.26| 9.31| 9.66| 11.0|  8.9| 8.11| 8.52| 8.79|12.46|10.73|12.49|13.51|127.07|  7.53|
|Residential and o...| 5.19| 5.53| 5.73| 5.54| 6.51| 5.94|  4.7| 6.08|  6.6| 6.39| 6.57| 6.27| 6.49| 77.54|   4.6|
|Manufacturing Ind...| 4.59| 3.26| 3.73| 4.35| 4.51| 5.24| 4.83|  3.8| 5.69| 5.33| 3.39|  5.7| 5.98|  60.4|  3.58|
| Road transportation| 4.29| 4.04| 4.13| 4.53| 4.65| 4.65| 4.98| 4.92| 4.37| 4.9

**Here, I think some countries are not segregating the sectors and that’s why we have the total row indicating the total emission in this year in this country which would have been better if they tried to have it segregated for better analysis sector wise.**

In [101]:
display(Markdown("TOP20**'Year-Wise' Most to lowest year emissions** by Country (in Megatons):"))
NEWDF.select("COUNTRY_NAME",year("date").alias("Year"),"VALUE")\
                     .groupBy("Year","COUNTRY_NAME") \
                     .agg((sum("VALUE")/1000000).alias("sumEmission(Mton)"), \
                          (sum("VALUE")/sumCO2Emission[0][0]*100).alias("Ratio"))\
                     .select("Year","COUNTRY_NAME",round("sumEmission(Mton)",2).alias("sumEmission(Mton)"), \
                             round("Ratio",2).alias("Ratio%")) \
                     .sort(["Year","Ratio%"], ascending=False) \
                     .show()
                     

TOP20**'Year-Wise' Most to lowest year emissions** by Country (in Megatons):

+----+--------------------+-----------------+------+
|Year|        COUNTRY_NAME|sumEmission(Mton)|Ratio%|
+----+--------------------+-----------------+------+
|2012|               China|            20.89|  1.24|
|2012|       United States|             9.51|  0.56|
|2012|               India|             5.95|  0.35|
|2012|  Russian Federation|             3.64|  0.22|
|2012|               Japan|             2.69|  0.16|
|2012|              Brazil|             2.02|  0.12|
|2012|             Germany|             1.77|   0.1|
|2012|           Indonesia|             1.45|  0.09|
|2012|       Int. Shipping|             1.22|  0.07|
|2012|              Canada|             1.24|  0.07|
|2012|Iran, Islamic Rep...|              1.2|  0.07|
|2012|             Nigeria|             1.17|  0.07|
|2012|  Korea, Republic of|             1.13|  0.07|
|2012|       Int. Aviation|             0.97|  0.06|
|2012|      United Kingdom|             0.99|  0.06|
|2012|        Saudi Arabia|             0.93| 

# Section4 (Summary)


***In Conclusion,***, it can be seen that **CO2** (KN.A2, KN.A3) are both the most type that is affecting the whole earth, both together are formulating **97.4%** of the overall emissions on earth for the period 2000 till 2012 but it can be seen from the above table that CO2 in both types are decreased dramatically to half of the percentage recorded in 2000, however, there is a slight increase in **CH4** which is more dangerous even with small doses to our earth, and to really imagine how CH4 is problematic to earth just imagine that its impact is **34 times** greater than CO2 over a 100-year period, according to the latest IPCC Assessment Report.
Country wise, **China, US, India, Russia and Brazil**  are the most countries that produces CO2 on earth with almost **44%** of the over all emission in the period from 2000 till 2012 which will need to work on reducing more there emissions by searching for another source of energy as they are affecting not only their area but also all earth.
Lastly, united states of America were at 1st place in emission till **2004** as **China took the lead till now in polluting the whole world**, there is also something happened in 2009 as there were a big drop by almost **50%** in all countries and I think this is related to the economic crisis happened in **2008**.

# Section5 (Resources)

MyDataSet:

https://datasource.kapsarc.org/explore/dataset/global-emissions-by-country-2013/export/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6Imdsb2JhbC1lbWlzc2lvbnMtYnktY291bnRyeS0yMDEzIiwib3B0aW9ucyI6e319LCJjaGFydHMiOlt7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJsaW5lIiwiZnVuYyI6IkFWRyIsInlBeGlzIjoidmFsdWUiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjNjZjMmE1In1dLCJ4QXhpcyI6ImRhdGUiLCJtYXhwb2ludHMiOiIiLCJ0aW1lc2NhbGUiOiJ5ZWFyIiwic29ydCI6IiJ9XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D

Why Methane Matters: <br>
https://unfccc.int/news/new-methane-signs-underline-urgency-to-reverse-emissions


The Secret History of Lead

https://www.thenation.com/article/archive/secret-history-lead/
<br>





#### Clearing all dataframes and Deleting all jobs and then stop spark context

In [102]:
NEWDF.unpersist()
subDF.unpersist()
COUNTRY_NAMEDF.unpersist()
COUNTRY_NOTESDF.unpersist()
COUNTRY_REGIONIDDF.unpersist()
INDICATOR_NAMEDF.unpersist()

DataFrame[INDICATOR_NAME: string, Total: bigint]

In [103]:
sc.cancelAllJobs()

In [104]:
spark.stop()
sc.stop()